# UNR - FCEIA 
## Tecnicatura Universitaria en Programación 
### NLP: Trabajo Práctico N°1 

---

**Integrantes**
- López Ceratto, Julieta : L-3311/1
- Crenna, Giuliano : C-7438/1

# Importamos librerías necesarias

In [32]:
import os
import pandas as pd
import pickle
from typing import List, Dict, Any
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.neighbors import NearestNeighbors
import numpy as np
import tensorflow_text
import spacy
from sklearn.decomposition import PCA
import plotly.express as px

In [33]:
import warnings
warnings.filterwarnings("ignore")

# Carga de datasets

In [34]:
JUEGOS_PATH: str = os.path.join(os.getcwd(), 'data', 'bgg_database.csv')
PELICULAS_PATH: str = os.path.join(os.getcwd(), 'data', 'IMDB-Movie-Data.csv')
LIBROS_PATH: str = os.path.join(os.getcwd(), 'data', 'dataset_libros.csv')

In [35]:
dataset_juegos: pd.DataFrame = pd.read_csv(JUEGOS_PATH)
dataset_peliculas: pd.DataFrame = pd.read_csv(PELICULAS_PATH)
dataset_libros: pd.DataFrame = pd.read_csv(LIBROS_PATH)

# Modelo de Análisis de Sentimientos

Creo un dataset sencillo para entrenar al clasificador.

In [37]:
ESTADOS_ANIMO_PATH: str = os.path.join(os.getcwd(), 'data', 'estados_de_animo.csv')

In [38]:
df_estados_de_animo: pd.DataFrame = pd.read_csv(ESTADOS_ANIMO_PATH)

X: pd.Series = df_estados_de_animo['prompt']
y: pd.Series = df_estados_de_animo['estado_animo']

Hacemos un split de los datos y creamos un pipeline de trabajo utilizando el clasificador **MultinomialNB**.

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

modelo_animo = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [40]:
modelo_animo.fit(X_train, y_train)

y_pred = modelo_animo.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      Alegre       1.00      0.96      0.98        79
 Melancólico       1.00      1.00      1.00        65
 Ni fu ni fa       0.96      1.00      0.98        76

    accuracy                           0.99       220
   macro avg       0.99      0.99      0.99       220
weighted avg       0.99      0.99      0.99       220



Creamos una función para clasificar el prompt del usuario.

In [41]:
def clasificar_animo(prompt_usuario: str) -> str:
    estado_animo_predicho = modelo_animo.predict([prompt_usuario])[0]
    
    return estado_animo_predicho

In [42]:
nuevo_prompt = "La vida no significa nada"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Ni fu ni fa


In [43]:
nuevo_prompt = "Me siento feliz"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Alegre


In [44]:
nuevo_prompt = "El vacio se siente en mi"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Melancólico


Exporto el modelo

In [45]:
# ESTADO_ANIMO_MODEL_PATH: str = os.path.join(os.getcwd(), 'models', 'modelo_estado_animo.pickle')

# pickle.dump(modelo_animo, open(ESTADO_ANIMO_MODEL_PATH, 'wb'))

# Análisis Datasets

## Análisis Dataset Juegos
Como Vemos en el dataset de juegos no se presentan datos nulos.

In [46]:
dataset_juegos.isna().sum()

rank                0
game_name           0
game_href           0
geek_rating         0
avg_rating          0
num_voters          0
description         0
yearpublished       0
minplayers          0
maxplayers          0
minplaytime         0
maxplaytime         0
minage              0
avgweight           0
best_num_players    0
designers           0
mechanics           0
categories          0
dtype: int64

## Análisis Dataset Películas

In [47]:
dataset_peliculas.isna().sum()

Rank                  0
Title                 0
Genre                 0
Description           0
Director              0
Actors                0
Year                  0
Runtime (Minutes)     0
Rating                0
Votes                 0
Revenue (Millions)    0
Metascore             0
dtype: int64

## Dataset Libros
De los 5984 libros dentro del dataset, solo 53 tiene titulo secundario y solo 2796 libros tienen un autor registrado.

In [48]:
dataset_libros.shape

(5986, 5)

In [49]:
dataset_libros['Resumen'].str.split().str.len().sum()

899588

Como tenemos problemas de cómputo al hacer embeddings con casi 6000 libros, lo que se traduce en aproximadasmente 9000 palabras en los resúmenes; reducimos el dataset de libros al tamaño del de películas (1000). Además, nos aseguramos que de este dataset resumido sólo formen parte libros que tienen descripción.

In [50]:
dataset_libros_resumido = dataset_libros[dataset_libros['Resumen']!= 'Resumen no encontrado']

Además, sacamos títulos repetidos.

In [51]:
dataset_libros_resumido.drop_duplicates(subset='Titulo Principal', inplace= True)

In [52]:
dataset_libros_resumido = dataset_libros_resumido.sample(n = 1000, random_state= 42)

In [53]:
dataset_libros_resumido.reset_index(drop=True, inplace=True)

In [54]:
dataset_peliculas.shape

(1000, 12)

In [55]:
dataset_libros.isna().sum()

Titulo Principal     0
Titulo Secundario    0
Autor                0
N° Ref               0
Resumen              0
dtype: int64

# Apartado 1

## Implementación modelo KNN con encoder de tensorflow

In [56]:
# Cargar Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
def crear_embeding():
    '''
    Crea dataset de embedings para entrenar el modelo y realizar posterior búsqueda
    en los resultados

    Detalles del proceso:
    -----------
    - Se agregan las columnas 'índice' y 'tipo' de cada fila en los 3 df (libros, juegos y películas),
        que corresponden al índice en dicho df y el tipo (si es juego, libro o película); de esta manera
        se diferenciarán en el dataset final yse podrán rastrear los embeddings.
    - Se agrega una columna 'frase_embeding', la cual concatena la información que se considera necesaria hacer
        embedding.
    - Se genera un df con los embeding por cada fila en cada df a partir de la columna anterior.
    - se agregan las columnas index y tipo al df embeding de cada tipo.
    - Se concatenan los df embeding de los tres tipos y se lo guarda en './data/embedings_totales.csv' como un
        archivo .csv.
    '''
    ###Agrega index y tipo a los dataset para luego recuperar el vecino más cercano
    ### por índice.
    dataset_juegos['index'] = [i for i in dataset_juegos.index]
    dataset_juegos['tipo'] = 'juego'
    dataset_libros_resumido['index'] = [i for i in dataset_libros_resumido.index]
    dataset_libros_resumido['tipo'] = 'libro'
    dataset_peliculas['index'] = [i for i in dataset_peliculas.index]
    dataset_peliculas['tipo'] = 'pelicula'

    ###Agrega una columna 'frase_embedding' que junta la información que se utilizará para entrenar
    ### al modelo.
    
    # Para dataset_juegos
    dataset_juegos['frase_embeding'] = dataset_juegos.apply(
        lambda row: f"{row['description']}, tipo juego, {row['maxplayers']}", axis=1)

    # Para dataset_peliculas
    dataset_peliculas['frase_embeding'] = dataset_peliculas.apply(
        lambda row: f"{row['Description']}, tipo pelicula, {row['Genre']}", axis=1)

    # Para dataset_libros_resumido
    dataset_libros_resumido['frase_embeding'] = dataset_libros_resumido.apply(
        lambda row: f"{row['Resumen']}, tipo libro, {row['Autor']}", axis=1)
    
    # Generar embeddings para cada conjunto de datos
    embeding_juegos = embed(dataset_juegos['frase_embeding']).numpy()
    embeding_libros = embed(dataset_libros_resumido['frase_embeding']).numpy()
    embeding_peliculas = embed(dataset_peliculas['frase_embeding']).numpy()

    # Crear DataFrames para los embeddings
    embeding_juegos_df = pd.DataFrame(embeding_juegos)
    embeding_libros_df = pd.DataFrame(embeding_libros)
    embeding_peliculas_df = pd.DataFrame(embeding_peliculas)

    # Añadir el índice y tipo a los DataFrames de embeddings
    embeding_juegos_df['index'] = dataset_juegos['index']
    embeding_libros_df['index'] = dataset_libros_resumido['index']
    embeding_peliculas_df['index'] = dataset_peliculas['index']
    embeding_juegos_df['tipo'] = dataset_juegos['tipo']
    embeding_libros_df['tipo'] = dataset_libros_resumido['tipo']
    embeding_peliculas_df['tipo'] = dataset_peliculas['tipo'] 

    # Concatenar todos los embeddings
    embedings_totales = np.concatenate([embeding_juegos, embeding_libros, embeding_peliculas])
    df_embedings_totales = pd.concat([embeding_juegos_df, embeding_libros_df, embeding_peliculas_df])
    
    # Guardar el DataFrame de embeddings en un archivo CSV
    df_embedings_totales.to_csv('./data/embedings_totales.csv', index=False)


Creamos los embedings necesarios para entrenar el modelo.

In [ ]:
## Se comenta la función ya que no es necesario volver a ejecutarla puesto que
## el df de embedings se guardo localmente en su primera ejecución.
#crear_embeding()
df_embedings_totales = pd.read_csv('./data/embedings_totales.csv')

## Cramos modelo KNN de recomendación.

Creamos el modelo con 5 vecinos más cercanos ya que es lo que se considera más apto a la hora de recomendar; podrían también ser 3.

In [70]:
n_neighbors = 5
modelor_recomendador = make_pipeline(NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm='brute'))
modelor_recomendador.fit(df_embedings_totales.drop(columns=['index', 'tipo']))

Pipeline(steps=[('nearestneighbors',
                 NearestNeighbors(algorithm='brute', metric='cosine'))])

Guardamos el modelo.

In [61]:
RECOMENDADOR_MODEL_PATH: str = os.path.join(os.getcwd(), 'models', 'modelo_recomendador.pickle')

#pickle.dump(modelor_recomendador, open(RECOMENDADOR_MODEL_PATH, 'wb'))

Definimos función que devuelve los 5 vecinos más cercanos a partir de una frase.

In [ ]:
def que_hacer (consulta : str):
    '''
    Devuelve 5 recomendaciones más acordes a la consulta

    Parámetros:
    -----------
    **consulta [str]:**
        string con la consulta del usuario compuesta por el estado de animo clasificado
        y una frase relacionada a lo que quiere.
    
    Detalles del proceso:
    -----------
    - Se hace un embeding de la consulta.
    - Con dicho embeding se busca los 5 vecinos más cercanos mediante el modelo_recomendador.
    - Se extrae de cada vecino el índice de la columna 'index' de df_embedings_totales el cual se corresponde
      al índice del vecino en su df original y el valor que le corresponde en la columna 'tipo' el cual indica de que 
      df original proviene.
    - Según el tipo (pelicula, libro o juego), busca el nombre del vecino mediante el índice en su df original.
    - Finalmente imprime los nombres de los 5 vecinos más cercanos, en conjunto con su tipo y su distancia / semejanza.

    '''
    consulta = embed(consulta).numpy()

    # Realizar la búsqueda de los vecinos más cercanos
    distances, indices = modelor_recomendador[0].kneighbors(consulta)

    for j in range(n_neighbors):
        # Obtenemos el índice del vecino más cercano
        idx = indices[0][j]
        i = df_embedings_totales['index'].iloc[idx]
        dataset = df_embedings_totales['tipo'].iloc[idx]

        # Dependiendo del dataset, accedemos al DataFrame correspondiente
        if dataset == 'juego':
            vecino = dataset_juegos['game_name'].iloc[i]
        elif dataset == 'libro':
            vecino = dataset_libros_resumido['Titulo Principal'].iloc[i]
        elif dataset == 'pelicula':
            vecino = dataset_peliculas['Title'].iloc[i]
        
        # Imprimir el vecino y la distancia
        print(f"Vecino {j + 1}: {vecino} - Distancia: {distances[0][j]:.4f} - {dataset}")

    print("-" * 40)

# Usuario.

Definimos función User la cual le solicita al usuario su estado de ánimo y una frase realcionada a su interés.

In [ ]:
def user():
    '''
    Solicita dos frases al usuario y llama a la función 'que_hacer'.

    Detalles del proceso:
    -----------
    - Soliciata al usuario que explique como se siente hoy.
    - Clasifica la frase anterior para extraer el ánimo.
    - Solicita al usuario qué está buscando y lo concatena con su ánimo.
    - Utiliza la concatenación anterior para pasar como parámetro a la función 'que_hacer'.
    
    '''
    frase_animo = input('¿Cómo se siente hoy?')
    animo_user = modelo_animo.predict(frase_animo)
    user_prompt = f"{input('ingrese qué está buscando')}, {animo_user}"
    que_hacer(user_prompt)

In [69]:
user()

Vecino 1: The Complete Poetical Works of Edgar Allan Poe                                                          - Distancia: 0.6412 - libro
Vecino 2: The Raven                                                                                               - Distancia: 0.6632 - libro
Vecino 3: The gardener                                                                                            - Distancia: 0.6700 - libro
Vecino 4: The Works of Edgar Allan Poe, The Raven Edition                                                       - Distancia: 0.6708 - libro
Vecino 5: Poems                                                                                                   - Distancia: 0.6854 - libro
----------------------------------------


# PCA para visualizar distribución.

In [ ]:
# Función para plotear embeddings en 3D
def plotear_embeddings_3d(df_embeddings: pd.DataFrame , juegos: pd.DataFrame, libros: pd.DataFrame, peliculas: pd.DataFrame):
    """
    ## Grafica los embeddings en 3D coloreados por tipo y muestra el nombre al pasar el cursor.\n
    
    Parámetros:
    -----------
    **df_embeddings [DataFrame]:**
        Dataframe que contiene los embeddings de los 3 dataframes (juegos, libros y películas).\n
    **juegos [DataFrame]:**
        Dataframe original de juegos.\n
    **libros [DataFrame]:**
        Dataframe original de libros.\n
    **peliculas [DataFrame]:**
        Dataframe original de peliculas.\n

    -----------
    ## Detalles del proceso:\n
    -----------
    - Se generan 3 componentes principales a partir de df_embeddings.\n
    - Según el tipo (juego, libro, película), se agrega el nombre correspondiente.\n
    - Se plotea una gráfica en 3D diferenciada por tipo y que al pasar el cursor
    muestra el nombre del elemento.\n

    """
    # Reducimos la dimensionalidad a 3 componentes para visualización
    pca = PCA(n_components=3)
    embeddings_3d = pca.fit_transform(df_embeddings.iloc[:, :-2])  # Eliminamos las columnas 'tipo' e 'index' de los datos
    df_embeddings_plot = df_embeddings.copy()

    # Añadimos las columnas de las componentes reducidas
    df_embeddings_plot['PCA1'] = embeddings_3d[:, 0]
    df_embeddings_plot['PCA2'] = embeddings_3d[:, 1]
    df_embeddings_plot['PCA3'] = embeddings_3d[:, 2]

    # Añadimos la columna de nombre según el tipo
    nombres = []
    for i, row in df_embeddings_plot.iterrows():
        if row['tipo'] == 'juego':
            nombres.append(juegos.loc[row['index'], 'game_name'])
        elif row['tipo'] == 'libro':
            nombres.append(libros.loc[row['index'], 'Titulo Principal'])
        elif row['tipo'] == 'pelicula':
            nombres.append(peliculas.loc[row['index'], 'Title'])
    df_embeddings_plot['nombre'] = nombres

    # Creamos el gráfico 3D interactivo
    fig = px.scatter_3d(
        df_embeddings_plot,
        x='PCA1',
        y='PCA2',
        z='PCA3',
        color='tipo',
        hover_name='nombre',
        title="Embeddings 3D coloreados por tipo con nombres interactivos"
    )
    fig.show()


In [68]:
plotear_embeddings_3d(df_embedings_totales, dataset_juegos,dataset_libros_resumido, dataset_peliculas)